In [1]:
from backtrader import plot
import backtrader as bt
import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
from backtrader.utils import num2date,date2num
import rba_tools.backtest.rba_backtrader_set as rbs
import plotly.graph_objects as go
import plotly.express as px
import rba_tools.backtest.backtrader_extensions.plot as rbsplot
import importlib

In [14]:
importlib.reload(rbsplot)

<module 'rba_tools.backtest.backtrader_extensions.plot' from 'c:\\users\\avery\\documents\\github\\rba_tools_clone\\src\\rba_tools\\backtest\\backtrader_extensions\\plot.py'>

In [3]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [4]:
cerebro = bt.Cerebro(runonce=False)

# Add a strategy
cerebro.addstrategy(MaCrossStrategy)
#cerebro.addstrategy(rbsstrat.ConsecutiveBarsTest)
#cerebro.addstrategy(rbsstrat.StopLimitEntryStrategy)

#get a datapuller
puller = rba.retriever.get_crypto_data.DataPuller.kraken_puller()

#symbol and date range
symbol = 'ETH/USD'
from_date = '1/1/20'
to_date = '6/30/20'

# Get a pandas dataframe
dataframe = puller.fetch_df(symbol, 'h', from_date, to_date)

# Pass it to the backtrader datafeed and add it to the cerebro
data = bt.feeds.PandasData(dataname=dataframe,
                            nocase=True,
                            )
#cerebro.adddata(data)
cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=240)

cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=1440)

# Run over everything
back = cerebro.run()

# Plot the result
#myp = cerebro.plot(numfigs=1, style='bar')

In [5]:
strategy = back[0]
from backtrader import plot
plotter = plot.Plot()
plotter

In [6]:
plotter.sortdataindicators(strategy)

In [7]:
vars(plotter)

{'params': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x1f20d1ad790>,
 'p': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x1f20d1ad790>,
 'dplotstop': [<backtrader.observers.broker.Broker at 0x1f273d33c40>,
 'dplotsup': defaultdict(list, {}),
 'dplotsdown': defaultdict(list,
             {<backtrader.indicators.sma.SMA at 0x1f2721cb3d0>: [<backtrader.indicators.crossover.CrossOver at 0x1f2722087c0>]}),
 'dplotsover': defaultdict(list,
             {<backtrader.feeds.pandafeed.PandasData at 0x1f2721e96a0>: [<backtrader.observers.buysell.BuySell at 0x1f273d6f280>,
              <backtrader.feed.DataClone at 0x1f2721cb160>: [<backtrader.observers.buysell.BuySell at 0x1f273d6f520>]})}

In [8]:
rbsplot.rbs_sort_indicators(strategy)

{'dplotstop': [<backtrader.observers.broker.Broker at 0x1f273d33c40>,
 'dplotsup': defaultdict(list, {}),
 'dplotsdown': defaultdict(list,
             {<backtrader.indicators.sma.SMA at 0x1f2721cb3d0>: [<backtrader.indicators.crossover.CrossOver at 0x1f2722087c0>]}),
 'dplotsover': defaultdict(list,
             {<backtrader.feeds.pandafeed.PandasData at 0x1f2721e96a0>: [<backtrader.observers.buysell.BuySell at 0x1f273d6f280>,
              <backtrader.feed.DataClone at 0x1f2721cb160>: [<backtrader.observers.buysell.BuySell at 0x1f273d6f520>]})}

In [9]:
plotter.plot(strategy)

[<Figure size 640x480 with 7 Axes>]

In [10]:
vars(plotter)

{'params': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x1f20d1ad790>,
 'p': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x1f20d1ad790>,
 'dplotstop': [<backtrader.observers.broker.Broker at 0x1f273d33c40>,
 'dplotsup': defaultdict(list,
             {<backtrader.feeds.pandafeed.PandasData at 0x1f2721e96a0>: [],
              <backtrader.observers.buysell.BuySell at 0x1f273d6f280>: [],
              <backtrader.indicators.sma.SMA at 0x1f2721cb3d0>: [],
              <backtrader.indicators.sma.SMA at 0x1f272208af0>: [],
              <backtrader.indicators.crossover.CrossOver at 0x1f2722087c0>: [],
              <backtrader.feed.DataClone at 0x1f2721cb160>: [],
              <backtrader.observers.buysell.BuySell at 0x1f273d6f520>: []}),
 'dplotsdown': defaultdict(list,
             {<backtrader.indicators.sma.SMA at 0x1f2721cb3d0>: [<backtrader.indicators.crossover.CrossOver at 0x1f2722087c0>],
              <backtrader.observers.buysell.BuySell at 0x1f273d6f280>: [],
    

In [12]:
rbsplot.get_ohlcv_data_from_data(strategy.data0)

,Open,High,Low,Close,Volume
2020-01-01 00:00:00,128.66,128.66,128.19,128.42,515.780816
2020-01-01 04:00:00,128.33,130.58,128.30,129.67,1857.032786
2020-01-01 08:00:00,129.67,130.25,129.60,129.81,1191.872012
2020-01-01 12:00:00,130.03,131.50,129.95,131.32,939.921635
2020-01-01 16:00:00,131.31,132.79,131.01,132.68,2201.396240
...,...,...,...,...,...
2020-06-30 08:00:00,226.39,227.15,225.77,226.05,4074.983230
2020-06-30 12:00:00,226.01,228.01,223.51,224.76,8255.857590
2020-06-30 16:00:00,224.52,226.73,222.80,225.75,12605.506344
2020-06-30 20:00:00,225.86,225.97,224.51,225.47,2980.591000


In [13]:
rbsplot.get_ohlcv_data_from_data(strategy.data1)

,Open,High,Low,Close,Volume
2020-01-01,128.66,128.66,128.19,128.42,515.780816
2020-01-02,128.33,132.86,128.30,130.19,13163.487459
2020-01-03,130.06,130.06,126.00,126.71,26250.065941
2020-01-04,126.69,134.72,125.40,133.58,50313.895117
2020-01-05,133.36,135.91,132.58,134.78,18000.273569
...,...,...,...,...,...
2020-06-27,232.93,233.19,226.86,229.30,41971.814456
2020-06-28,229.51,230.89,215.90,219.35,60857.542028
2020-06-29,219.35,227.57,217.80,224.61,38486.237540
2020-06-30,224.63,229.82,221.19,227.76,43663.927594


In [15]:
res = rbsplot.my_plot(strategy)
res

ValueError: Length of values (1093) does not match length of index (183)